# Python内存管理

Python内存管理的总机制：计数机制为主，标记-清除和分代收集为辅。

## 1. 对象引用计数

### 1.1 对象存储

Python对象存储机制：双向环状链表。

链表的结构体包含的元数据有：

- 上一个对象的地址
- 下一个对象的地址
- 对象的类型
- 引用的个数（**引用计数器**）
- 值类型：对象的值
- 集合类型：元素的个数

**前四个为所有对象共有的元数据类型。**

### 1.2 引用计数器

在C源码中，每个对象中有一个`ob_refcnt`为引用计数器。

引用计数器的值可以使用`sys.getrefcount(item)`来获取。获取时会临时使该值+1，且不总是真实值，但是可以反映计数变化。

引用计数器的初始值为1，当引用一次的时候+1，删除一次的时候-1，如代码块1.1。

引用计数器为0时：

1. 将对象从链表中删除；
2. 对象销毁，归还内存。


In [2]:
# Block 1.1

import sys

a = 2.513
print(sys.getrefcount(a) - 1)

b = a
print(sys.getrefcount(a) - 1)  # a的引用计数器增加
print(sys.getrefcount(b) - 1)

del b
print(sys.getrefcount(a) - 1)  # a的引用计数器减少

2
3
3
2


## 2. 标记-清除机制

### 2.1 循环引用的内存泄露问题

如下代码逻辑：

```python
v1 = [1, 2, 3]
v2 = [4, 5, 6]

v1.append(v2)  # v2的引用计数器为2
v2.append(v1)  # v1的引用计数器为2

del v1  # 引用计数器-1，为1
del v2  # 引用计数器-1，为1
```

就会引发一个问题：两个变量的引用次数为1，不会被删除，但是指向它们的变量消失导致不能操作->内存泄漏

### 2.2 标记清除

目的：解决引用计数器机制在循环引用中的问题。

实现：在底层增加一个链表，用于存储可能存在循环引用的对象。一定条件下检查是否有循环引用的问题，定期检查，使循环引用的引用计数器-1，若得到0则直接清除。

## 3. 分代回收

机制：形成三个链表，分别为0代、1代、2代：

- 0代：对象个数达到700个后进行一次扫描；
- 1代：0代扫描10次时，扫描一次；
- 2代：1代扫描10次时，扫描一次。

## 4. 缓存机制

Python使用缓存机制来节省内存：

### 4.1 池

对于常用数据，Python使用池进行预存，在使用时不会开辟新内存，而是直接指向池中的地址。如代码块4.1。

池中的数据为从-5到256的int类型数值。

### 4.2 free_list机制

对引用计数器置0的情况，将对象添加到free_list链表中进行缓存。再开辟对象时，不使用新的内存空间，而是优先使用free_list的内存空间。

对不同的类型有不同的机制和容量：

- int类型：不基于free_list，而是使用数据池；
- float类型：容量为100个；
- str类型：维护unicode_latin1[256]，包含所有的ASCII字符，使用时不再创建；
- list类型：最多80个对象；
- tuple类型：元组元素数为1到20，索引用于标记元素数，每种元素数最多2000个元素；
- dict类型：容量为80。


In [4]:
# Block 4.1

v1 = 3
v2 = 3
print(id(v1))
print(id(v2))  # v1和v2的地址相同

v3 = 666
v4 = 666
print(id(v3))
print(id(v4))  # 池中没有此数值，地址不同

9771880
9771880
140537918606256
140537918606224


In [6]:
# Block 4.2

m = 6.13
print(id(m))
del m
n = 7.28
print(id(n))

140537988543376
140537988542608
